# 电商案例之营销推荐

## 读入数据

In [1]:
import pandas as pd
import numpy as np

In [3]:
orders = pd.read_csv("data/orders.csv")
items = pd.read_csv("data/Items_orders.csv")
itemProps = pd.read_csv("data/Items_attribute.csv",encoding='gbk')

In [4]:
orders.head()

,订单编号,买家会员名,买家支付宝账号,买家应付货款,买家应付邮费,买家支付积分,总金额,返点积分,买家实际支付金额,买家实际支付积分,...,是否代付,定金排名,修改后的sku,修改后的收货地址,异常信息,天猫卡券抵扣,集分宝抵扣,是否是O2O交易,退款金额,预约门店
0,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,24270488269081200,2163,AC870BA5860,15.70,5.0,0,20.70,0,20.70,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,21402600386365500,375,AC7574B65A0,7.90,5.0,0,12.90,0,12.90,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,21398820349555700,2618,A807C90766A,4.81,5.0,0,9.81,0,9.81,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,21446781606162100,2012,A505588565B,23.92,5.0,0,28.92,0,28.92,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [5]:
orders.shape

(3989, 46)

In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3989 entries, 0 to 3988
Data columns (total 46 columns):
订单编号        3989 non-null int64
买家会员名       3989 non-null int64
买家支付宝账号     3988 non-null object
买家应付货款      3989 non-null float64
买家应付邮费      3989 non-null float64
买家支付积分      3989 non-null int64
总金额         3989 non-null float64
返点积分        3989 non-null int64
买家实际支付金额    3989 non-null float64
买家实际支付积分    3989 non-null int64
订单状态        3989 non-null object
买家留言        384 non-null object
收货人姓名       3989 non-null int64
收货地址        3989 non-null object
运送方式        3989 non-null object
联系电话        142 non-null object
联系手机        3986 non-null object
订单创建时间      3989 non-null object
订单付款时间      3989 non-null object
宝贝标题        3989 non-null object
宝贝种类        3989 non-null int64
物流单号        3988 non-null object
物流公司        3988 non-null object
订单备注        460 non-null object
宝贝总数量       3989 non-null int64
店铺Id        3989 non-null int64
店铺名称        3989 non-null int64
订单关闭原因     

In [3]:
items.head()

,订单编号,标题,价格,购买数量,外部系统编号,商品属性,套餐信息,备注,订单状态,商家编码
0,21407300627014900,发光玩具批发光纤手指灯闪光夜市热卖货源儿童玩具地摊义乌厂家,0.58,12,WY013-2SZD0426,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426
1,21407300627014900,特价5号AA普通干电池 电动玩具配件 厂家直销批,1.00,20,HT-5H0094,NaN,NaN,NaN,交易成功,HT-5H0094
2,21407300627014900,大号泡泡棒沙滩小铲子工具泡泡枪公园吹泡泡户外亲子游戏玩具热卖,1.20,16,GC018005,NaN,NaN,NaN,交易成功,GC018005
3,21407300627014900,特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发,1.00,10,HT-7H0093,NaN,NaN,NaN,交易成功,HT-7H0093
4,21407300627014900,儿童沙滩玩具水枪 宝宝玩水玩具户外洗澡游泳漂流戏大号水枪批发,6.79,1,GC080003,套餐类型：标准配置;颜色分类：33000蓝色水枪,NaN,NaN,交易成功,GC080003


In [7]:
items.shape

(21897, 10)

In [8]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21897 entries, 0 to 21896
Data columns (total 10 columns):
订单编号      21897 non-null int64
标题        21897 non-null object
价格        21897 non-null float64
购买数量      21897 non-null int64
外部系统编号    21897 non-null object
商品属性      12636 non-null object
套餐信息      0 non-null float64
备注        130 non-null object
订单状态      21897 non-null object
商家编码      21897 non-null object
dtypes: float64(2), int64(2), object(6)
memory usage: 1.7+ MB


In [9]:
itemProps.head()

,宝贝ID,标题,价格,玩具类型,适用年龄,品牌
0,537396783238,创意新款回力小车惯性坦克 军事儿童玩具模型地摊货源玩具车批发,8.90,塑胶玩具,"3岁,4岁,5岁,6岁",3
1,36286235128,2017热卖大号仿真惯性挖土机儿童益智礼品创意义乌地摊货玩具批发,3.90,其它玩具,"3岁,4岁,5岁,6岁",3
2,35722000205,创意发光球闪光透明发光水晶弹力球儿童小玩具 夜市地摊货源批发,1.65,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁,11岁,12岁",3
3,550659732532,新款创意六一儿童节礼物音乐投影电动枪夜市地摊货源批发男孩玩具,9.90,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁,11岁,12岁,13岁,14岁",3
4,531877266868,发条玩具批发上链卡通动物青蛙儿童礼物宝宝玩具经典80后益智地摊,1.85,其它玩具,"3岁,4岁",3


In [10]:
itemProps.shape

(288, 6)

In [11]:
itemProps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 6 columns):
宝贝ID    288 non-null int64
标题      288 non-null object
价格      288 non-null float64
玩具类型    252 non-null object
适用年龄    284 non-null object
品牌      288 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 13.6+ KB


## 将数据合并成一张表

In [12]:
orders_items = pd.merge(orders,items,on="订单编号")

In [13]:
orders_items.head()

,订单编号,买家会员名,买家支付宝账号,买家应付货款,买家应付邮费,买家支付积分,总金额,返点积分,买家实际支付金额,买家实际支付积分,...,预约门店,标题,价格,购买数量,外部系统编号,商品属性,套餐信息,备注,订单状态_y,商家编码
0,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,发光玩具批发光纤手指灯闪光夜市热卖货源儿童玩具地摊义乌厂家,0.58,12,WY013-2SZD0426,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426
1,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,特价5号AA普通干电池 电动玩具配件 厂家直销批,1.00,20,HT-5H0094,NaN,NaN,NaN,交易成功,HT-5H0094
2,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,大号泡泡棒沙滩小铲子工具泡泡枪公园吹泡泡户外亲子游戏玩具热卖,1.20,16,GC018005,NaN,NaN,NaN,交易成功,GC018005
3,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发,1.00,10,HT-7H0093,NaN,NaN,NaN,交易成功,HT-7H0093
4,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,儿童沙滩玩具水枪 宝宝玩水玩具户外洗澡游泳漂流戏大号水枪批发,6.79,1,GC080003,套餐类型：标准配置;颜色分类：33000蓝色水枪,NaN,NaN,交易成功,GC080003


In [14]:
orders_items.shape

(21897, 55)

In [15]:
orders_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21897 entries, 0 to 21896
Data columns (total 55 columns):
订单编号        21897 non-null int64
买家会员名       21897 non-null int64
买家支付宝账号     21887 non-null object
买家应付货款      21897 non-null float64
买家应付邮费      21897 non-null float64
买家支付积分      21897 non-null int64
总金额         21897 non-null float64
返点积分        21897 non-null int64
买家实际支付金额    21897 non-null float64
买家实际支付积分    21897 non-null int64
订单状态_x      21897 non-null object
买家留言        2240 non-null object
收货人姓名       21897 non-null int64
收货地址        21897 non-null object
运送方式        21897 non-null object
联系电话        825 non-null object
联系手机        21890 non-null object
订单创建时间      21897 non-null object
订单付款时间      21897 non-null object
宝贝标题        21897 non-null object
宝贝种类        21897 non-null int64
物流单号        21896 non-null object
物流公司        21896 non-null object
订单备注        3248 non-null object
宝贝总数量       21897 non-null int64
店铺Id        21897 non-null int64
店铺名称        2189

In [20]:
sum(orders_items.loc[:,['订单状态_x']].values != orders_items.loc[:,['订单状态_y']].values)

array([73])

In [22]:
orders_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21897 entries, 0 to 21896
Data columns (total 55 columns):
订单编号        21897 non-null int64
买家会员名       21897 non-null int64
买家支付宝账号     21887 non-null object
买家应付货款      21897 non-null float64
买家应付邮费      21897 non-null float64
买家支付积分      21897 non-null int64
总金额         21897 non-null float64
返点积分        21897 non-null int64
买家实际支付金额    21897 non-null float64
买家实际支付积分    21897 non-null int64
订单状态_x      21897 non-null object
买家留言        2240 non-null object
收货人姓名       21897 non-null int64
收货地址        21897 non-null object
运送方式        21897 non-null object
联系电话        825 non-null object
联系手机        21890 non-null object
订单创建时间      21897 non-null object
订单付款时间      21897 non-null object
宝贝标题        21897 non-null object
宝贝种类        21897 non-null int64
物流单号        21896 non-null object
物流公司        21896 non-null object
订单备注        3248 non-null object
宝贝总数量       21897 non-null int64
店铺Id        21897 non-null int64
店铺名称        2189

In [25]:
orders_items.loc[3,'标题']

'特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发'

In [26]:
itemProps.loc[5,'标题']

'儿童拍拍球玩具幼儿园球类宝宝新款充气球小孩无毒西瓜皮球批'

In [27]:
orders_items_props = pd.merge(orders_items,itemProps,on="标题")

In [28]:
orders_items_props.head()

,订单编号,买家会员名,买家支付宝账号,买家应付货款,买家应付邮费,买家支付积分,总金额,返点积分,买家实际支付金额,买家实际支付积分,...,商品属性,套餐信息,备注,订单状态_y,商家编码,宝贝ID,价格_y,玩具类型,适用年龄,品牌
0,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3
1,24043728806509300,882,A5BBABBB0B7,173.10,0.0,0,173.10,0,173.10,0,...,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3
2,24043728806509300,882,A5BBABBB0B7,173.10,0.0,0,173.10,0,173.10,0,...,颜色分类：大号,NaN,NaN,交易成功,WY013-1SZD0425,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3
3,20885882368182100,279,csongCAA6C,82.73,0.0,0,82.73,0,82.73,0,...,颜色分类：大号,NaN,NaN,交易成功,WY013-1SZD0425,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3
4,20885882368182100,279,csongCAA6C,82.73,0.0,0,82.73,0,82.73,0,...,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3


In [29]:
orders_items_props.shape

(19943, 60)

In [30]:
orders_items_props.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19943 entries, 0 to 19942
Data columns (total 60 columns):
订单编号        19943 non-null int64
买家会员名       19943 non-null int64
买家支付宝账号     19940 non-null object
买家应付货款      19943 non-null float64
买家应付邮费      19943 non-null float64
买家支付积分      19943 non-null int64
总金额         19943 non-null float64
返点积分        19943 non-null int64
买家实际支付金额    19943 non-null float64
买家实际支付积分    19943 non-null int64
订单状态_x      19943 non-null object
买家留言        2030 non-null object
收货人姓名       19943 non-null int64
收货地址        19943 non-null object
运送方式        19943 non-null object
联系电话        739 non-null object
联系手机        19936 non-null object
订单创建时间      19943 non-null object
订单付款时间      19943 non-null object
宝贝标题        19943 non-null object
宝贝种类        19943 non-null int64
物流单号        19942 non-null object
物流公司        19942 non-null object
订单备注        2986 non-null object
宝贝总数量       19943 non-null int64
店铺Id        19943 non-null int64
店铺名称        1994

## 构建关系矩阵

In [31]:
result = orders_items_props.loc[:,["买家会员名","宝贝ID"]]

In [32]:
result.head()

,买家会员名,宝贝ID
0,1425,530449665002
1,882,530449665002
2,882,530449665002
3,279,530449665002
4,279,530449665002


In [33]:
result.shape

(19943, 2)

In [36]:
result["购买次数"] = 0 #添加一列并初始化为0

In [37]:
result.head()

,买家会员名,宝贝ID,购买次数
0,1425,530449665002,0
1,882,530449665002,0
2,882,530449665002,0
3,279,530449665002,0
4,279,530449665002,0


In [45]:
result.groupby(["买家会员名","宝贝ID"],as_index=False).count().head(2)

,买家会员名,宝贝ID,购买次数
0,0,42577833473,1
1,1,536728628605,1


In [41]:
freq = result.groupby(["买家会员名","宝贝ID"]).count().reset_index()

In [42]:
freq.head()

,买家会员名,宝贝ID,购买次数
0,0,42577833473,1
1,1,536728628605,1
2,1,545516801138,1
3,1,547644315780,1
4,1,550735773284,1


In [46]:
freq= freq.pivot(index="买家会员名",columns="宝贝ID",values="购买次数")

In [47]:
freq.head()

宝贝ID,35721027449,35721723963,35722000205,35722333869,35722423659,35750823403,35753244214,35754637865,35797606083,35798309577,...,551081926272,551091071810,551091439907,551135995408,551248890640,551625132527,551675713112,551715398940,552007245556,552028581381
买家会员名,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
freq.shape

(3318, 269)

In [49]:
freqMatrix = freq.fillna(0).values
freqMatrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 计算物品相似度

In [50]:
from sklearn.metrics.pairwise import cosine_similarity
item_similar=cosine_similarity(freqMatrix.T)

In [52]:
pd.DataFrame(item_similar)

,0,1,2,3,4,5,6,7,8,9,...,259,260,261,262,263,264,265,266,267,268
0,1.000000,0.101630,0.146662,0.127454,0.000000,0.000000,0.000000,0.095019,0.044076,0.000000,...,0.068519,0.000000,0.083918,0.000000,0.000000,0.059339,0.000000,0.000000,0.000000,0.000000
1,0.101630,1.000000,0.130841,0.088007,0.076338,0.023473,0.053231,0.074796,0.130107,0.128446,...,0.000000,0.064931,0.036699,0.031782,0.028427,0.038925,0.055048,0.000000,0.000000,0.000000
2,0.146662,0.130841,1.000000,0.093601,0.033930,0.037261,0.014083,0.059366,0.050486,0.069907,...,0.000000,0.025768,0.023302,0.000000,0.045125,0.012358,0.000000,0.000000,0.000000,0.000000
3,0.127454,0.088007,0.093601,1.000000,0.033255,0.051127,0.083739,0.054306,0.062977,0.013323,...,0.032634,0.035358,0.039968,0.000000,0.030959,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.076338,0.033930,0.033255,1.000000,0.000000,0.022350,0.000000,0.036418,0.046225,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.023473,0.037261,0.051127,0.000000,1.000000,0.000000,0.048280,0.055989,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.053231,0.014083,0.083739,0.022350,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.095019,0.074796,0.059366,0.054306,0.000000,0.048280,0.000000,1.000000,0.089205,0.000000,...,0.000000,0.033389,0.000000,0.000000,0.000000,0.080064,0.000000,0.000000,0.000000,0.160128
8,0.044076,0.130107,0.050486,0.062977,0.036418,0.055989,0.000000,0.089205,1.000000,0.218844,...,0.000000,0.038720,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.128446,0.069907,0.013323,0.046225,0.000000,0.000000,0.000000,0.218844,1.000000,...,0.000000,0.049147,0.055556,0.096225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [53]:
#构建预测函数
def predict(similar):
    user_cnt = freqMatrix.shape[0]#用户数3318
    item_cnt = freqMatrix.shape[1]#商品数269
    pred = np.zeros((user_cnt,item_cnt)) #初始化一个全零矩阵
    for uid in range(user_cnt):
        for iid in range(item_cnt):
            if freqMatrix[uid,iid] == 0:
                #print (uid,iid)
                pred[uid,iid] = Recommendation_s(uid,iid,similar)
    return pred

In [54]:
#构建一个物品的推荐
def Recommendation_s(uid,iid,similar,k=10):
    score = 0
    weight = 0
    uid_action = freqMatrix[uid,:]      #用户uid 对所有商品的行为评分  
    iid_action = freqMatrix[:,iid]      #物品iid 得到的所有用户评分  
    

    iid_sim = similar[iid,:]      #商品iid 对所有商品的相似度    
    sim_indexs = np.argsort(iid_sim)[-(k+1):-1]  #最相似的k个物品的index（除了自己）
    iid_i_mean = np.sum(iid_action)/iid_action[iid_action!=0].size
    for j in sim_indexs :
        if uid_action[j]!=0:
            iid_j_action = freqMatrix[:,j]
            iid_j_mean = np.sum(iid_j_action)/iid_j_action[iid_j_action!=0].size
            score += iid_sim[j]*(uid_action[j]-iid_j_mean)
            weight += abs(iid_sim[j])

    if weight==0:  
        return 0
    else:
        return iid_i_mean + score/float(weight)    

In [55]:
user_prediction = predict(item_similar)

In [57]:
user_prediction.shape

(3318, 269)

In [58]:
pd.DataFrame(user_prediction)

,0,1,2,3,4,5,6,7,8,9,...,259,260,261,262,263,264,265,266,267,268
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.891026,0.000000,1.458673,1.195238,0.884504,0.000000,1.687500,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.980952,0.000000,0.000000,0.000000,0.000000,1.000000
2,0.891026,0.000000,1.458673,0.000000,0.884504,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.891026,0.000000,1.458673,0.000000,0.884504,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.891026,0.000000,1.145696,0.000000,0.884504,0.000000,1.473214,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.708306,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.972619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.807512,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.993914,0.000000,1.164286,0.372375,0.000000,0.000000,0.000000,0.974684,0.000000,...,0.000000,0.000000,0.950000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [59]:
def get_recom(prediction,k=5):
    recom_df = pd.DataFrame(prediction,columns=freq.columns,index=freq.index)  
    recom_df = recom_df.stack().reset_index()   
    recom_df.rename(columns={0:"推荐指数"},inplace=True)    
    grouped = recom_df.groupby("买家会员名")   
    topk = grouped.apply(get_topk,k=k)
    topk = topk.drop(["买家会员名"],axis=1)
    topk.index = topk.index.droplevel(1)   
    topk.reset_index(inplace=True)
    return topk

In [60]:
def get_topk(group,k):
    return group.sort_values("推荐指数",ascending=False)[:k]

In [62]:
#看了又看
user_recom = get_recom(user_prediction,5)
user_recom 

,买家会员名,宝贝ID,推荐指数
0,0,544016559367,2.158102
1,0,537396783238,1.945519
2,0,544115359956,1.830060
3,0,546275765548,0.866250
4,0,550715341924,0.783523
5,1,35753244214,1.687500
6,1,35722000205,1.458673
7,1,520310825412,1.292639
8,1,35722333869,1.195238
9,1,527475911875,1.144947
